<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/mockup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')

In [7]:
import json
json.loads(data['billing'].iloc[1])

[{'billing_initial': '1032.00',
  'billing_discount': '103.00',
  'simb_billing_code': '1.1.1(3)',
  'billing_net_amount': '929.00',
  'local_billing_code': '50',
  'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'},
 {'billing_initial': '650',
  'billing_discount': '0',
  'simb_billing_code': '1.1.12',
  'billing_net_amount': '650.00',
  'local_billing_code': '240',
  'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'},
 {'billing_initial': '310.00',
  'billing_discount': '0.00',
  'simb_billing_code': '1.1.14(2)',
  'billing_net_amount': '310.00',
  'local_billing_code': '280',
  'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'},
 {'billing_initial': '429.00',
  'billing_discount': '0.00',
  'simb_billing_code': '1.1.2(1)',
  'billing_net_amount': '429.00',
  'local_billing_code': '70',
  'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'},
 {'billing_initial': '1200',
  'billing_discount': '0',
  'simb_billing_code': '1.

In [8]:
data.columns

Index(['id', 'seq', 'created_at', 'updated_at', 'deleted_at',
       'service_setting', 'illnes_type', 'policy_type', 'cid', 'cid_type',
       'claim_status', 'claim_status_desc', 'ref_id', 'transaction_uid',
       'transaction_no', 'is_follow_up', 'further_claim_id',
       'electronic_signature', 'accident_cause_over45_days', 'admit_date_time',
       'admit_date_time_txt', 'dsc_date_time', 'dsc_date_time_txt',
       'chief_complaint', 'is_procedure', 'icd9', 'dx_free_text', 'coma_score',
       'claim_from_motorcycle', 'visit_with_alcohol_or_drug',
       'alcohol_related', 'addiction_related', 'pregnant', 'total_bill_amount',
       'total_bill_amount_txt', 'patient', 'vital_sign', 'doctor',
       'order_item', 'billing', 'attach_doc_list', 'diagnosis', 'response',
       'check_eligible_id', 'insurance', 'insurance_id', 'hospital_code',
       'hospital_id', 'claim_no', 'discharge_date', 'discharge_date_txt',
       'member_id', 'further_claim', 'patient_registration_id',
    

In [41]:
json.loads(data['doctor'].iloc[0])

[{'role': 'OWNER',
  'title': 'นพ.',
  'license': '34296',
  'lastname': 'ศรีอัษฎาพร',
  'firstname': 'เอกภักดิ์ '}]

In [22]:
data = data[data['claim_status'].isin(['Approve','Decline'])]

In [47]:
icd10_df = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/icd10 - ICD-10 v.2019.csv')

In [48]:
icd10_df[['ICD-10 Code','Descriptions (EN)']].head()

,ICD-10 Code,Descriptions (EN)
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A009,"Cholera, unspecified"
3,A010,Typhoid fever
4,A011,Paratyphoid fever A


In [53]:
mockup = data[data['vital_sign']!='[]'][['hospital_code','cid','patient','diagnosis','icd9','vital_sign','doctor']]
mockup.shape

(4659, 7)

In [49]:
!pip -q install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.4 MB/s eta 0:00:00


In [57]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker(['en_US', 'th_TH'])

# Configuration
num_rows = 5000

# Sample data for each field
hospital_codes = [11787, 40765, 11583, 11558, 11768, 23781, 41829, 11864, 14683,
                  21673, 11720, 11771, 14591, 11543, 41345, 24258, 11963, 24257,
                  13766, 11645, 14354, 14550, 11766, 23924, 11878, 12026, 41556,
                  12188, 12226, 12159, 13779, 41674, 14641, 14534, 12139, 11765,
                  28875, 42025, 41590, 23875, 11919, 11730, 11537, 10814, 41569,
                  11845, 11708, 11644, 11648, 10686, 44109, 10827, 10734, 10663,
                  10679, 42167, 11874, 42770, 10821, 44353, 42015, 11037, 11656,
                  10695, 12230, 42141, 10849, 11359, 11621, 12212, 13778, 11920,
                  10674, 10723, 10671, 10998, 10822, 10667, 11360, 10832]

# ICD-10 code and diagnosis name samples
diagnosis_samples = [
    {'icd10': row['ICD-10 Code'], 'dx_name': row['Descriptions (EN)']}
    for _, row in icd10_df.iterrows()
]

# Generate data
data = []
for _ in range(num_rows):
    hospital_code = random.choice(hospital_codes)
    cid = fake.unique.random_number(digits=13)
    patient = {
        'hn': f"{random.randint(10, 99)}-{random.randint(10000, 99999)}",
        'dob': fake.date_of_birth().isoformat(),
        'gender': random.choice(['M', 'F']),
        'last_name': fake.last_name(),
        'first_name': fake.first_name(),
        'title_name': random.choice(['นาย', 'นาง', 'นางสาว']),
        'patient_registration_id': fake.uuid4()
    }
    diagnosis = [random.choice(diagnosis_samples) for _ in range(random.randint(1, 3))]
    vital_sign = [{
        'heart_rate': str(random.randint(60, 100)),
        'pain_score': str(random.randint(0, 10)),
        'systolic_bp': f"{random.uniform(90, 180):.2f}",
        'temperature': f"{random.uniform(36.0, 37.5):.2f}",
        'diastolic_bp': f"{random.uniform(60, 120):.2f}",
        'respiratory_rate': str(random.randint(12, 20)),
        'oxygen_saturation': f"{random.uniform(95, 100):.2f}",
        'vital_sign_entry_datetime': fake.date_time_this_year().strftime("%Y-%m-%d %H:%M")
    }]
    doctor = [{
        'role': 'OWNER',
        'title': 'นพ.',
        'license': str(random.randint(10000, 99999)),
        'lastname': fake.last_name(),
        'firstname': fake.first_name()
    }]
    # claim_status = random.choice(['Approve', 'Decline'])
    # insurance = random.choice(['aia', 'mtl', 'fwd', 'tli', 'oli', 'tmlth', 'plt', 'bla'])

    # Append row to data
    data.append({
        'hospital_code': hospital_code,
        'cid': cid,
        'patient': patient,
        'diagnosis': diagnosis,
        'vital_sign': vital_sign,
        'doctor': doctor,
        # 'claim_status': claim_status,
        # 'insurance': insurance,
    })

In [58]:
# Create DataFrame
mock1 = pd.DataFrame(data)
mock1

,hospital_code,cid,patient,diagnosis,vital_sign,doctor
0,44109,4060189518756,"{'hn': '81-49373', 'dob': '1995-01-01', 'gende...","[{'icd10': 'C761', 'dx_name': 'Malignant neopl...","[{'heart_rate': '93', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
1,13778,1493695634508,"{'hn': '54-70002', 'dob': '1915-12-20', 'gende...","[{'icd10': 'E275', 'dx_name': 'Adrenomedullary...","[{'heart_rate': '91', 'pain_score': '9', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
2,11648,8790943954778,"{'hn': '97-67018', 'dob': '1930-10-02', 'gende...","[{'icd10': 'X1508', 'dx_name': 'Contact with h...","[{'heart_rate': '61', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
3,13766,1963496354067,"{'hn': '28-17171', 'dob': '1932-04-18', 'gende...","[{'icd10': 'M1985', 'dx_name': 'Other specifie...","[{'heart_rate': '60', 'pain_score': '7', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
4,11537,8245834798036,"{'hn': '28-40112', 'dob': '2014-01-07', 'gende...","[{'icd10': 'M413', 'dx_name': 'Thoracogenic sc...","[{'heart_rate': '83', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
...,...,...,...,...,...,...
4995,21673,7106378781908,"{'hn': '50-69333', 'dob': '2021-11-09', 'gende...","[{'icd10': 'W025', 'dx_name': 'Fall involving ...","[{'heart_rate': '63', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
4996,14683,3075422365741,"{'hn': '84-25502', 'dob': '1920-11-03', 'gende...","[{'icd10': 'X1242', 'dx_name': 'Contact with o...","[{'heart_rate': '66', 'pain_score': '8', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
4997,11768,4600511398215,"{'hn': '70-87542', 'dob': '2001-12-04', 'gende...","[{'icd10': 'V1249', 'dx_name': 'Pedal cyclist ...","[{'heart_rate': '91', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."
4998,11766,870383699051,"{'hn': '40-35250', 'dob': '2002-11-14', 'gende...","[{'icd10': 'V926', 'dx_name': 'Water-transport...","[{'heart_rate': '78', 'pain_score': '10', 'sys...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ..."


In [59]:
mock1['patient'].iloc[0]

{'hn': '81-49373',
 'dob': '1995-01-01',
 'gender': 'M',
 'last_name': 'หิรัญสาลี',
 'first_name': 'วิชชากร',
 'title_name': 'นาย',
 'patient_registration_id': '2364c5b4-f207-408c-af5d-77864191d93f'}

In [60]:
mock1['vital_sign'].iloc[0]

[{'heart_rate': '93',
  'pain_score': '0',
  'systolic_bp': '110.88',
  'temperature': '37.48',
  'diastolic_bp': '67.44',
  'respiratory_rate': '15',
  'oxygen_saturation': '99.65',
  'vital_sign_entry_datetime': '2024-10-02 16:32'}]

In [61]:
mock1['doctor'].iloc[0]

[{'role': 'OWNER',
  'title': 'นพ.',
  'license': '72116',
  'lastname': 'Lee',
  'firstname': 'โอภาส'}]

In [62]:
mock1['diagnosis'].iloc[0]

[{'icd10': 'C761', 'dx_name': 'Malignant neoplasm of thorax'},
 {'icd10': 'W5642',
  'dx_name': 'Contact with marine animal: at street & highway: while work for income'},
 {'icd10': 'P152', 'dx_name': 'Sternomastoid injury due to birth injury'}]

In [63]:
mock1.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/patient.csv')